##Analyzing a all_vars_for_zeroinf_analysis

- This Python notebook demonstrates creating an ML Pipeline to preprocess a dataset, train a Machine Learning model, and make predictions.

- Data: The dataset contains all_vars_for_zeroinf_analysis  

- Goal: We want to learn to predict confirmed deaths  from information such as 

- Approach: We will use Spark ML Pipelines, which help users piece together parts of a workflow such as feature processing and model training. We will also demonstrate model selection (a.k.a. hyperparameter tuning) using Cross Validation in order to fine-tune and improve our ML model.

In [2]:
from pyspark.sql.functions import log

In [3]:
%sh ls /dbfs/FileStore/tables/

all_vars_for_analysis.csv
all_vars_for_zeroinf_analysis.csv
Feature_Importances_predicting_deaths-d1408.csv
Final_Features_for_analysis-2c971.csv
Final_Features_for_analysis.csv
health.csv
health_factors_behab.csv
health_factors.csv
Health_Professional_Shortage_Areas__Primary_Care__by_County-c0119.csv
housing.csv
Lockdown_Join_with_normaliztion_factor-ed399.csv
merged.csv
merged_dataset.xlsx
merged_emp.csv
merged_source.csv
States_Lock_down_Lengh-8d3af.csv
States_Lockdown_Lengh.csv
testing_by_state-971a2.csv
Train_X.csv
us_counties.csv
us_counties-e349f.csv
X_test.csv
X_train.csv
y_test.csv
y_train.csv

In [4]:
# File location and type
file_location = "/FileStore/tables/all_vars_for_zeroinf_analysis.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

## import all_vars_for_zeroinf_analysis  file 
# The applied options are for CSV files. For other file types, these will be ignored.
zeroinf_analysis = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [5]:
display(zeroinf_analysis)

county,confirmed_cases,confirmed_deaths,state,length_of_lockdown,cases,deaths,POP_ESTIMATE_2018,total_state_pop,Active.Physicians.per.100.000.Population..2018..AAMC.,Total.Active.Patient.Care.Physicians.per.100.000.Population..2018..AAMC.,Active.Primary.Care.Physicians.per.100.000.Population..2018..AAMC.,Active.Patient.Care.Primary.Care.Physicians.per.100.000.Population..2018..AAMC.,Active.General.Surgeons.per.100.000.Population..2018..AAMC.,Active.Patient.Care.General.Surgeons.per.100.000.Population..2018..AAMC.,Percentage.of.Active.Physicians.Who.Are.Female..2018..AAMC.,Percentage.of.Active.Physicians.Who.Are.International.Medical.Graduates..IMGs...2018..AAMC.,Percentage.of.Active.Physicians.Who.Are.Age.60.or.Older..2018..AAMC.,MD.and.DO.Student.Enrollment.per.100.000.Population..AY.2018.2019..AAMC.,Student.Enrollment.at.Public.MD.and.DO.Schools.per.100.000.Population..AY.2018.2019..AAMC.,Percentage.Change.in.Student.Enrollment.at.MD.and.DO.Schools..2008.2018..AAMC.,Percentage.of.MD.Students.Matriculating.In.State..AY.2018.2019..AAMC.,Total.Residents.Fellows.in.ACGME.Programs.per.100.000.Population.as.of.December.31..2018..AAMC.,Total.Residents.Fellows.in.Primary.Care.ACGME.Programs.per.100.000.Population.as.of.Dec..31..2018..AAMC.,Percentage.of.Residents.in.ACGME.Programs.Who.Are.IMGs.as.of.December.31..2018..AAMC.,Ratio.of.Residents.and.Fellows..GME..to.Medical.Students..UME...AY.2017.2018..AAMC.,Percent.Change.in.Residents.and.Fellows.in.ACGME.Accredited.Programs..2008.2018..AAMC.,Percentage.of.Physicians.Retained.in.State.from.Undergraduate.Medical.Education..UME...2018..AAMC.,All.Specialties..AAMC.,State.Local.Government.hospital.beds.per.1000.people..2019.,Non.profit.hospital.beds.per.1000.people..2019.,For.profit.hospital.beds.per.1000.people..2019.,Total.hospital.beds.per.1000.people..2019.,Total.nurse.practitioners..2019.,Total.physician.assistants..2019.,Total.Hospitals..2019.,Total.Primary.Care.Physicians..2019.,Surgery.specialists..2019.,Emergency.Medicine.specialists..2019.,Total.Specialist.Physicians..2019.,ICU.Beds,pop_fraction,Length.of.Life.rank,Quality.of.Life.rank,Health.Behaviors.rank,Clinical.Care.rank,Social...Economic.Factors.rank,Physical.Environment.rank,Adult.smoking.percentage,Adult.obesity.percentage,Excessive.drinking.percentage,Population.per.sq.mile,House.per.sq.mile,Share.of.Tests.with.Positive.COVID.19.Results,Number.of.Tests.with.Results.per.1.000.Population
Autauga,93,4,Alabama,26,1.672631787,0.071941152,55601,9775742,217.1,196.7,77.2,71.2,7.6,6.9,28.5,18.1,34.4,49.1,22.8,136.8,82.1,29.9,11.9,21.9,0.6,18.4,48.4,2.17149757,1.4,0.8,0.9,3.1,0.519039885,0.109454607,0.020663393,1.199499741,0.15425939,0.095133444,1.297497418,0.107911728,0.00568765,5.0,11.0,9.0,13.0,7.0,53.0,18,33,15,91.8,37.2,0.077853,27.8
Baldwin,231,7,Alabama,26,1.059526103,0.032106852,218022,9775742,217.1,196.7,77.2,71.2,7.6,6.9,28.5,18.1,34.4,49.1,22.8,136.8,82.1,29.9,11.9,21.9,0.6,18.4,48.4,2.17149757,1.4,0.8,0.9,3.1,0.519039885,0.109454607,0.020663393,1.199499741,0.15425939,0.095133444,1.297497418,0.233921347,0.022302348,3.0,2.0,5.0,6.0,3.0,5.0,17,31,18,114.6,65.5,0.077853,27.8
Barbour,69,1,Alabama,26,2.773200434,0.040191311,24881,9775742,217.1,196.7,77.2,71.2,7.6,6.9,28.5,18.1,34.4,49.1,22.8,136.8,82.1,29.9,11.9,21.9,0.6,18.4,48.4,2.17149757,1.4,0.8,0.9,3.1,0.519039885,0.109454607,0.020663393,1.199499741,0.15425939,0.095133444,1.297497418,0.200956553,0.002545178,23.0,55.0,54.0,41.0,60.0,28.0,22,42,13,31.0,13.4,0.077853,27.8
Bibb,46,1,Alabama,26,2.053571429,0.044642857,22400,9775742,217.1,196.7,77.2,71.2,7.6,6.9,28.5,18.1,34.4,49.1,22.8,136.8,82.1,29.9,11.9,21.9,0.6,18.4,48.4,2.17149757,1.4,0.8,0.9,3.1,0.519039885,0.109454607,0.020663393,1.199499742,0.15425939,0.095133444,1.297497417,0.0,0.002291386,49.0,14.0,32.0,18.0,43.0,22.0,19,38,16,36.8,14.4,0.077853,27.8
Blount,45,0,Alabama,26,0.778008299,0.0,57840,9775742,217.1,196.7,77.2,71.2,7.6,6.9,28.5,18.1,34.4,49.1,22.8,136.8,82.1,29.9,11.9,21.9,0.6,18.4,48.4,2.171497571,1.4,0

In [6]:
zeroinf_analysis.count()

Out[221]: 2890

In [7]:
zeroinf_analysis.dtypes

Out[222]: [('county', 'string'),
 ('confirmed_cases', 'int'),
 ('confirmed_deaths', 'int'),
 ('state', 'string'),
 ('length_of_lockdown', 'int'),
 ('cases', 'double'),
 ('deaths', 'double'),
 ('POP_ESTIMATE_2018', 'int'),
 ('total_state_pop', 'int'),
 ('Active.Physicians.per.100.000.Population..2018..AAMC.', 'double'),
 ('Total.Active.Patient.Care.Physicians.per.100.000.Population..2018..AAMC.',
 'double'),
 ('Active.Primary.Care.Physicians.per.100.000.Population..2018..AAMC.',
 'double'),
 ('Active.Patient.Care.Primary.Care.Physicians.per.100.000.Population..2018..AAMC.',
 'double'),
 ('Active.General.Surgeons.per.100.000.Population..2018..AAMC.', 'double'),
 ('Active.Patient.Care.General.Surgeons.per.100.000.Population..2018..AAMC.',
 'double'),
 ('Percentage.of.Active.Physicians.Who.Are.Female..2018..AAMC.', 'double'),
 ('Percentage.of.Active.Physicians.Who.Are.International.Medical.Graduates..IMGs...2018..AAMC.',
 'double'),
 ('Percentage.of.Active.Physicians.Who.Are.Age.60.or.Older..2018..AAMC.',
 'double'),
 ('MD.and.DO.Student.Enrollment.per.100.000.Population..AY.2018.2019..AAMC.',
 'double'),
 ('Student.Enrollment.at.Public.MD.and.DO.Schools.per.100.000.Population..AY.2018.2019..AAMC.',
 'double'),
 ('Percentage.Change.in.Student.Enrollment.at.MD.and.DO.Schools..2008.2018..AAMC.',
 'double'),
 ('Percentage.of.MD.Students.Matriculating.In.State..AY.2018.2019..AAMC.',
 'double'),
 ('Total.Residents.Fellows.in.ACGME.Programs.per.100.000.Population.as.of.December.31..2018..AAMC.',
 'double'),
 ('Total.Residents.Fellows.in.Primary.Care.ACGME.Programs.per.100.000.Population.as.of.Dec..31..2018..AAMC.',
 'double'),
 ('Percentage.of.Residents.in.ACGME.Programs.Who.Are.IMGs.as.of.December.31..2018..AAMC.',
 'double'),
 ('Ratio.of.Residents.and.Fellows..GME..to.Medical.Students..UME...AY.2017.2018..AAMC.',
 'double'),
 ('Percent.Change.in.Residents.and.Fellows.in.ACGME.Accredited.Programs..2008.2018..AAMC.',
 'double'),
 ('Percentage.of.Physicians.Retained.in.State.from.Undergraduate.Medical.Education..UME...2018..AAMC.',
 'double'),
 ('All.Specialties..AAMC.', 'double'),
 ('State.Local.Government.hospital.beds.per.1000.people..2019.', 'double'),
 ('Non.profit.hospital.beds.per.1000.people..2019.', 'double'),
 ('For.profit.hospital.beds.per.1000.people..2019.', 'double'),
 ('Total.hospital.beds.per.1000.people..2019.', 'double'),
 ('Total.nurse.practitioners..2019.', 'double'),
 ('Total.physician.assistants..2019.', 'double'),
 ('Total.Hospitals..2019.', 'double'),
 ('Total.Primary.Care.Physicians..2019.', 'double'),
 ('Surgery.specialists..2019.', 'double'),
 ('Emergency.Medicine.specialists..2019.', 'double'),
 ('Total.Specialist.Physicians..2019.', 'double'),
 ('ICU.Beds', 'double'),
 ('pop_fraction', 'double'),
 ('Length.of.Life.rank', 'double'),
 ('Quality.of.Life.rank', 'double'),
 ('Health.Behaviors.rank', 'double'),
 ('Clinical.Care.rank', 'double'),
 ('Social...Economic.Factors.rank', 'double'),
 ('Physical.Environment.rank', 'double'),
 ('Adult.smoking.percentage', 'int'),
 ('Adult.obesity.percentage', 'int'),
 ('Excessive.drinking.percentage', 'int'),
 ('Population.per.sq.mile', 'double'),
 ('House.per.sq.mile', 'double'),
 ('Share.of.Tests.with.Positive.COVID.19.Results', 'double'),
 ('Number.of.Tests.with.Results.per.1.000.Population', 'double')]

In [8]:
new_column_name_list= list(map(lambda x: x.replace(".", "_"), zeroinf_analysis.columns))

zeroinf_analysis = zeroinf_analysis.toDF(*new_column_name_list)

In [9]:
zeroinf_analysis.where( zeroinf_analysis['confirmed_deaths'].isNull() ).count()

Out[224]: 0

In [10]:
zeroinf_analysis.where( zeroinf_analysis['pop_fraction'].isNull() ).count()

Out[225]: 0

In [11]:
def count_nulls(df):
    null_counts = []          #make an empty list to hold our results
    for col in df.dtypes:     #iterate through the column data types we saw above, e.g. ('C0', 'bigint')
        cname = col[0]        #splits out the column name, e.g. 'C0'    
        ctype = col[1]        #splits out the column type, e.g. 'bigint'
        if ctype != 'string': #skip processing string columns for efficiency (can't have nulls)
            nulls = df.where( df[cname].isNull() ).count()
            result = tuple([cname, nulls])  #new tuple, (column name, null count)
            null_counts.append(result)      #put the new tuple in our result list
    return null_counts

null_counts = count_nulls(zeroinf_analysis)

In [12]:
null_counts

Out[227]: [('confirmed_cases', 0),
 ('confirmed_deaths', 0),
 ('length_of_lockdown', 0),
 ('cases', 0),
 ('deaths', 0),
 ('POP_ESTIMATE_2018', 0),
 ('total_state_pop', 0),
 ('Active_Physicians_per_100_000_Population__2018__AAMC_', 0),
 ('Total_Active_Patient_Care_Physicians_per_100_000_Population__2018__AAMC_',
 0),
 ('Active_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_', 0),
 ('Active_Patient_Care_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_',
 0),
 ('Active_General_Surgeons_per_100_000_Population__2018__AAMC_', 0),
 ('Active_Patient_Care_General_Surgeons_per_100_000_Population__2018__AAMC_',
 0),
 ('Percentage_of_Active_Physicians_Who_Are_Female__2018__AAMC_', 0),
 ('Percentage_of_Active_Physicians_Who_Are_International_Medical_Graduates__IMGs___2018__AAMC_',
 0),
 ('Percentage_of_Active_Physicians_Who_Are_Age_60_or_Older__2018__AAMC_', 0),
 ('MD_and_DO_Student_Enrollment_per_100_000_Population__AY_2018_2019__AAMC_',
 0),
 ('Student_Enrollment_at_Public_MD_and_DO_Schools_per_100_000_Population__AY_2018_2019__AAMC_',
 0),
 ('Percentage_Change_in_Student_Enrollment_at_MD_and_DO_Schools__2008_2018__AAMC_',
 0),
 ('Percentage_of_MD_Students_Matriculating_In_State__AY_2018_2019__AAMC_', 0),
 ('Total_Residents_Fellows_in_ACGME_Programs_per_100_000_Population_as_of_December_31__2018__AAMC_',
 0),
 ('Total_Residents_Fellows_in_Primary_Care_ACGME_Programs_per_100_000_Population_as_of_Dec__31__2018__AAMC_',
 0),
 ('Percentage_of_Residents_in_ACGME_Programs_Who_Are_IMGs_as_of_December_31__2018__AAMC_',
 0),
 ('Ratio_of_Residents_and_Fellows__GME__to_Medical_Students__UME___AY_2017_2018__AAMC_',
 0),
 ('Percent_Change_in_Residents_and_Fellows_in_ACGME_Accredited_Programs__2008_2018__AAMC_',
 0),
 ('Percentage_of_Physicians_Retained_in_State_from_Undergraduate_Medical_Education__UME___2018__AAMC_',
 0),
 ('All_Specialties__AAMC_', 0),
 ('State_Local_Government_hospital_beds_per_1000_people__2019_', 0),
 ('Non_profit_hospital_beds_per_1000_people__2019_', 0),
 ('For_profit_hospital_beds_per_1000_people__2019_', 0),
 ('Total_hospital_beds_per_1000_people__2019_', 0),
 ('Total_nurse_practitioners__2019_', 0),
 ('Total_physician_assistants__2019_', 0),
 ('Total_Hospitals__2019_', 0),
 ('Total_Primary_Care_Physicians__2019_', 0),
 ('Surgery_specialists__2019_', 0),
 ('Emergency_Medicine_specialists__2019_', 0),
 ('Total_Specialist_Physicians__2019_', 0),
 ('ICU_Beds', 0),
 ('pop_fraction', 0),
 ('Length_of_Life_rank', 0),
 ('Quality_of_Life_rank', 0),
 ('Health_Behaviors_rank', 0),
 ('Clinical_Care_rank', 0),
 ('Social___Economic_Factors_rank', 0),
 ('Physical_Environment_rank', 0),
 ('Adult_smoking_percentage', 0),
 ('Adult_obesity_percentage', 0),
 ('Excessive_drinking_percentage', 0),
 ('Population_per_sq_mile', 0),
 ('House_per_sq_mile', 0),
 ('Share_of_Tests_with_Positive_COVID_19_Results', 0),
 ('Number_of_Tests_with_Results_per_1_000_Population', 0)]

In [13]:
zeroinf_analysis_described = zeroinf_analysis.describe()
zeroinf_analysis_described.show()

+-------+---------+------------------+-----------------+-------+------------------+-----------------+-------------------+------------------+--------------------+-----------------------------------------------------+------------------------------------------------------------------------+------------------------------------------------------------------+-------------------------------------------------------------------------------+-----------------------------------------------------------+------------------------------------------------------------------------+-----------------------------------------------------------+-------------------------------------------------------------------------------------------+--------------------------------------------------------------------+------------------------------------------------------------------------+------------------------------------------------------------------------------------------+------------------------------------------------------------------------------+---------------------------------------------------------------------+-----------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+--------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------+----------------------+-----------------------------------------------------------+-----------------------------------------------+-----------------------------------------------+------------------------------------------+--------------------------------+---------------------------------+----------------------+------------------------------------+--------------------------+-------------------------------------+----------------------------------+-------------------+--------------------+-------------------+--------------------+---------------------+------------------+------------------------------+-------------------------+------------------------+------------------------+-----------------------------+----------------------+-----------------+---------------------------------------------+-------------------------------------------------+
summary| county| confirmed_cases| confirmed_deaths| state|length_of_lockdown| cases| deaths| POP_ESTIMATE_2018| total_state_pop|Active_Physicians_per_100_000_Population__2018__AAMC_|Total_Active_Patient_Care_Physicians_per_100_000_Population__2018__AAMC_|Active_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_|Active_Patient_Care_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_|Active_General_Surgeons_per_100_000_Population__2018__AAMC_|Active_Patient_Care_General_Surgeons_per_100_000_Population__2018__AAMC_|Percentage_of_Active_Physicians_Who_Are_Female__2018__AAMC_|Percentage_of_Active_Physicians_Who_Are_International_Medical_Graduates__IMGs___2018__AAMC_|Percentage_of_Active_Physicians_Who_Are_Age_60_or_Older__2018__AAMC_|MD_and_DO_Student_Enrollment_per_100_000_Population__AY_2018_2019__AAMC_|Student_Enrollment_at_Public_MD_and_DO_Schools_per_100_000_Population__AY_2018_2019__AAMC_|Percentage_Change_in_Student_Enrollment_at_MD_and_DO_Schools__2008_2018__AAMC_|Percentage_of_MD_Students_Matriculating_In_State__AY_2018_2019__AAMC_|Total_Residents_Fellows_in_ACGME_Programs_per_100_000_Population_as_of_December_31__2018__AAMC_|Total_Residents_Fellows_in_Primary_Care_ACGME_Programs_per_100_000_Population_as_of_Dec__31__2018__AAMC_|Percentage_of_Residents_in_ACGME_Programs_Who_Are_IMGs_as_of_December_31__2018__AAMC_|Ratio_of_Residents_and_Fellows__GME__to_Medical_Students__UME___AY_2017_2018__AAMC_|Percent_Change_in_Residents_and_Fellows_in_ACGME_Accre

In [14]:
from pyspark.sql.functions import skewness, kurtosis
from pyspark.sql.functions import var_pop, var_samp, stddev, stddev_pop, sumDistinct, ntile
zeroinf_analysis.select(skewness('confirmed_deaths')).show()


+--------------------------+
skewness(confirmed_deaths)|
+--------------------------+
 9.791804720559224|
+--------------------------+

In [15]:
from pyspark.sql import Row

columns = zeroinf_analysis_described.columns  #list of column names
funcs   = [skewness, kurtosis]  #list of functions we want to include (imported earlier)
fnames  = ['skew', 'kurtosis']  #a list of strings describing the functions in the same order

def new_item(func, column):
    """
    This function takes in an aggregation function and a column name, then applies the aggregation to the
    column, collects it and returns a value.  The value is in string format despite being a number, 
    because that matches the output of describe.
    """
    return str(zeroinf_analysis.select(func(column)).collect()[0][0])

new_data = []
for func, fname in zip(funcs, fnames):
    row_dict = {'summary':fname}  #each row object begins with an entry for "summary"
    for column in columns[1:]:
        row_dict[column] = new_item(func, column)
    new_data.append(Row(**row_dict))  #using ** tells Python to unpack the entries of the dictionary
    
print(new_data)

[Row(summary='skew', county='None', confirmed_cases='11.009453662781933', confirmed_deaths='9.791804720559224', state='None', length_of_lockdown='-0.5743978518845761', cases='9.726344345814274', deaths='5.02635252504621', POP_ESTIMATE_2018='13.73768721011897', total_state_pop='1.8060966426268625', Active_Physicians_per_100_000_Population__2018__AAMC_='2.0468748853313765', Total_Active_Patient_Care_Physicians_per_100_000_Population__2018__AAMC_='1.5192576017131847', Active_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_='1.306654476141262', Active_Patient_Care_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_='1.096630292190715', Active_General_Surgeons_per_100_000_Population__2018__AAMC_='2.006230069948525', Active_Patient_Care_General_Surgeons_per_100_000_Population__2018__AAMC_='1.65641771563768', Percentage_of_Active_Physicians_Who_Are_Female__2018__AAMC_='-0.2359653309914734', Percentage_of_Active_Physicians_Who_Are_International_Medical_Graduates__IMGs___2018__AAMC_='0.24678413845870986', Percentage_of_Active_Physicians_Who_Are_Age_60_or_Older__2018__AAMC_='0.5060181629157671', MD_and_DO_Student_Enrollment_per_100_000_Population__AY_2018_2019__AAMC_='2.4811040685325825', Student_Enrollment_at_Public_MD_and_DO_Schools_per_100_000_Population__AY_2018_2019__AAMC_='2.7731190701271093', Percentage_Change_in_Student_Enrollment_at_MD_and_DO_Schools__2008_2018__AAMC_='1.6427684491488883', Percentage_of_MD_Students_Matriculating_In_State__AY_2018_2019__AAMC_='-0.7317118973619294', Total_Residents_Fellows_in_ACGME_Programs_per_100_000_Population_as_of_December_31__2018__AAMC_='1.876532634631023', Total_Residents_Fellows_in_Primary_Care_ACGME_Programs_per_100_000_Population_as_of_Dec__31__2018__AAMC_='2.231936463898481', Percentage_of_Residents_in_ACGME_Programs_Who_Are_IMGs_as_of_December_31__2018__AAMC_='0.47600978309285336', Ratio_of_Residents_and_Fellows__GME__to_Medical_Students__UME___AY_2017_2018__AAMC_='0.5621054363577925', Percent_Change_in_Residents_and_Fellows_in_ACGME_Accredited_Programs__2008_2018__AAMC_='5.501828328967119', Percentage_of_Physicians_Retained_in_State_from_Undergraduate_Medical_Education__UME___2018__AAMC_='-0.1000231220302085', All_Specialties__AAMC_='2.0470977928344607', State_Local_Government_hospital_beds_per_1000_people__2019_='2.408457697241497', Non_profit_hospital_beds_per_1000_people__2019_='1.0993748666024383', For_profit_hospital_beds_per_1000_people__2019_='0.35968960732115685', Total_hospital_beds_per_1000_people__2019_='0.8223893999684955', Total_nurse_practitioners__2019_='0.7476873010234218', Total_physician_assistants__2019_='-0.026001829737361298', Total_Hospitals__2019_='1.4215019639257556', Total_Primary_Care_Physicians__2019_='1.4089144461352563', Surgery_specialists__2019_='1.3250169644535321', Emergency_Medicine_specialists__2019_='1.536501239473279', Total_Specialist_Physicians__2019_='2.034446003527229', ICU_Beds='43.5335293961527', pop_fraction='9.584592646351235', Length_of_Life_rank='1.7931523467720412', Quality_of_Life_rank='1.7272797185486595', Health_Behaviors_rank='1.7164754505547513', Clinical_Care_rank='1.6971082547615677', Social___Economic_Factors_rank='1.7412612938288288', Physical_Environment_rank='1.7102498563305553', Adult_smoking_percentage='0.7847910686933075', Adult_obesity_percentage='-0.1002967550395719', Excessive_drinking_percentage='0.16177195733968358', Population_per_sq_mile='11.066295533001421', House_per_sq_mile='11.836807578091502', Share_of_Tests_with_Positive_COVID_19_Results='0.9348694979526109', Number_of_Tests_with_Results_per_1_000_Population='1.918537174883447'), Row(summary='kurtosis', county='None', confirmed_cases='168.50484494872845', confirmed_deaths='116.55544719041936', state='None', length_of_lockdown='-0.20525975559433363', cases='165.59688093440127', deaths='38.55703177961991', POP_ESTIMATE_2018='310.565753869956', total_state_pop='2.826408484204448', Active_Physicians_per_100_000_Population__2018__AAMC_='14.28104

In [16]:
zeroinf_analysis_described.collect()

Out[231]: [Row(summary='count', county='2890', confirmed_cases='2890', confirmed_deaths='2890', state='2890', length_of_lockdown='2890', cases='2890', deaths='2890', POP_ESTIMATE_2018='2890', total_state_pop='2890', Active_Physicians_per_100_000_Population__2018__AAMC_='2890', Total_Active_Patient_Care_Physicians_per_100_000_Population__2018__AAMC_='2890', Active_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_='2890', Active_Patient_Care_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_='2890', Active_General_Surgeons_per_100_000_Population__2018__AAMC_='2890', Active_Patient_Care_General_Surgeons_per_100_000_Population__2018__AAMC_='2890', Percentage_of_Active_Physicians_Who_Are_Female__2018__AAMC_='2890', Percentage_of_Active_Physicians_Who_Are_International_Medical_Graduates__IMGs___2018__AAMC_='2890', Percentage_of_Active_Physicians_Who_Are_Age_60_or_Older__2018__AAMC_='2890', MD_and_DO_Student_Enrollment_per_100_000_Population__AY_2018_2019__AAMC_='2890', Student_Enrollment_at_Public_MD_and_DO_Schools_per_100_000_Population__AY_2018_2019__AAMC_='2890', Percentage_Change_in_Student_Enrollment_at_MD_and_DO_Schools__2008_2018__AAMC_='2890', Percentage_of_MD_Students_Matriculating_In_State__AY_2018_2019__AAMC_='2890', Total_Residents_Fellows_in_ACGME_Programs_per_100_000_Population_as_of_December_31__2018__AAMC_='2890', Total_Residents_Fellows_in_Primary_Care_ACGME_Programs_per_100_000_Population_as_of_Dec__31__2018__AAMC_='2890', Percentage_of_Residents_in_ACGME_Programs_Who_Are_IMGs_as_of_December_31__2018__AAMC_='2890', Ratio_of_Residents_and_Fellows__GME__to_Medical_Students__UME___AY_2017_2018__AAMC_='2890', Percent_Change_in_Residents_and_Fellows_in_ACGME_Accredited_Programs__2008_2018__AAMC_='2890', Percentage_of_Physicians_Retained_in_State_from_Undergraduate_Medical_Education__UME___2018__AAMC_='2890', All_Specialties__AAMC_='2890', State_Local_Government_hospital_beds_per_1000_people__2019_='2890', Non_profit_hospital_beds_per_1000_people__2019_='2890', For_profit_hospital_beds_per_1000_people__2019_='2890', Total_hospital_beds_per_1000_people__2019_='2890', Total_nurse_practitioners__2019_='2890', Total_physician_assistants__2019_='2890', Total_Hospitals__2019_='2890', Total_Primary_Care_Physicians__2019_='2890', Surgery_specialists__2019_='2890', Emergency_Medicine_specialists__2019_='2890', Total_Specialist_Physicians__2019_='2890', ICU_Beds='2890', pop_fraction='2890', Length_of_Life_rank='2890', Quality_of_Life_rank='2890', Health_Behaviors_rank='2890', Clinical_Care_rank='2890', Social___Economic_Factors_rank='2890', Physical_Environment_rank='2890', Adult_smoking_percentage='2890', Adult_obesity_percentage='2890', Excessive_drinking_percentage='2890', Population_per_sq_mile='2890', House_per_sq_mile='2890', Share_of_Tests_with_Positive_COVID_19_Results='2890', Number_of_Tests_with_Results_per_1_000_Population='2890'),
 Row(summary='mean', county=None, confirmed_cases='375.2373702422145', confirmed_deaths='19.05363321799308', state=None, length_of_lockdown='34.3', cases='2.37343276013253', deaths='0.0986733990647059', POP_ESTIMATE_2018='109859.56712802769', total_state_pop='1.8063285047058824E7', Active_Physicians_per_100_000_Population__2018__AAMC_='255.83636678200773', Total_Active_Patient_Care_Physicians_per_100_000_Population__2018__AAMC_='226.6442906574413', Active_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_='88.00505190311414', Active_Patient_Care_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_='80.04795847750937', Active_General_Surgeons_per_100_000_Population__2018__AAMC_='7.609515570934332', Active_Patient_Care_General_Surgeons_per_100_000_Population__2018__AAMC_='6.750311418685208', Percentage_of_Active_Physicians_Who_Are_Female__2018__AAMC_='33.87982698961921', Percentage_of_Active_Physicians_Who_Are_International_Medical_Graduates__IMGs___2018__AAMC_='20.962802768166316', Percentage_of_Active_Physicians_Who_Are_Age_60_or_Older__2018__AAMC_='31.21110

In [17]:
new_describe = sc.parallelize(new_data).toDF()           #turns the results from our loop into a dataframe
new_describe = new_describe.select(zeroinf_analysis_described.columns) #forces the columns into the same order

expanded_describe = zeroinf_analysis_described.unionAll(new_describe)  #merges the new stats with the original describe
expanded_describe.show()


+--------+---------+------------------+------------------+-------+--------------------+------------------+-------------------+------------------+--------------------+-----------------------------------------------------+------------------------------------------------------------------------+------------------------------------------------------------------+-------------------------------------------------------------------------------+-----------------------------------------------------------+------------------------------------------------------------------------+-----------------------------------------------------------+-------------------------------------------------------------------------------------------+--------------------------------------------------------------------+------------------------------------------------------------------------+------------------------------------------------------------------------------------------+------------------------------------------------------------------------------+---------------------------------------------------------------------+-----------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+--------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------+----------------------+-----------------------------------------------------------+-----------------------------------------------+-----------------------------------------------+------------------------------------------+--------------------------------+---------------------------------+----------------------+------------------------------------+--------------------------+-------------------------------------+----------------------------------+-------------------+--------------------+-------------------+--------------------+---------------------+------------------+------------------------------+-------------------------+------------------------+------------------------+-----------------------------+----------------------+------------------+---------------------------------------------+-------------------------------------------------+
 summary| county| confirmed_cases| confirmed_deaths| state| length_of_lockdown| cases| deaths| POP_ESTIMATE_2018| total_state_pop|Active_Physicians_per_100_000_Population__2018__AAMC_|Total_Active_Patient_Care_Physicians_per_100_000_Population__2018__AAMC_|Active_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_|Active_Patient_Care_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_|Active_General_Surgeons_per_100_000_Population__2018__AAMC_|Active_Patient_Care_General_Surgeons_per_100_000_Population__2018__AAMC_|Percentage_of_Active_Physicians_Who_Are_Female__2018__AAMC_|Percentage_of_Active_Physicians_Who_Are_International_Medical_Graduates__IMGs___2018__AAMC_|Percentage_of_Active_Physicians_Who_Are_Age_60_or_Older__2018__AAMC_|MD_and_DO_Student_Enrollment_per_100_000_Population__AY_2018_2019__AAMC_|Student_Enrollment_at_Public_MD_and_DO_Schools_per_100_000_Population__AY_2018_2019__AAMC_|Percentage_Change_in_Student_Enrollment_at_MD_and_DO_Schools__2008_2018__AAMC_|Percentage_of_MD_Students_Matriculating_In_State__AY_2018_2019__AAMC_|Total_Residents_Fellows_in_ACGME_Programs_per_100_000_Population_as_of_December_31__2018__AAMC_|Total_Residents_Fellows_in_Primary_Care_ACGME_Programs_per_100_000_Population_as_of_Dec__31__2018__AAMC_|Percentage_of_Residents_in_ACGME_Programs_Who_Are_IMGs_as_of_December_31__2018__AAMC_|Ratio_of_Residents_and_Fellows__GME__to_Medical_Students__UME___AY_2017_2018__AAMC_|Percent_Change_in_Residents_and_Fellows_in_ACG

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline #tells the Jupyter Notebook to display graphs inline (rather than in a separate window)

%matplotlib inline #tells the Jupyter Notebook to display graphs inline (rather than in a separate window) is not supported in Databricks.
Try using '%matplotlib inline' instead.

In [19]:
confirmed_deaths = zeroinf_analysis[['confirmed_deaths']].collect()
confirmed_cases = zeroinf_analysis[['confirmed_cases']].collect()

In [20]:
print(confirmed_deaths[:5])
print(confirmed_cases[:5])

[Row(confirmed_deaths=4), Row(confirmed_deaths=7), Row(confirmed_deaths=1), Row(confirmed_deaths=1), Row(confirmed_deaths=0)]
[Row(confirmed_cases=93), Row(confirmed_cases=231), Row(confirmed_cases=69), Row(confirmed_cases=46), Row(confirmed_cases=45)]

In [21]:
zeroinf_analysis.columns


Out[236]: ['county',
 'confirmed_cases',
 'confirmed_deaths',
 'state',
 'length_of_lockdown',
 'cases',
 'deaths',
 'POP_ESTIMATE_2018',
 'total_state_pop',
 'Active_Physicians_per_100_000_Population__2018__AAMC_',
 'Total_Active_Patient_Care_Physicians_per_100_000_Population__2018__AAMC_',
 'Active_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_',
 'Active_Patient_Care_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_',
 'Active_General_Surgeons_per_100_000_Population__2018__AAMC_',
 'Active_Patient_Care_General_Surgeons_per_100_000_Population__2018__AAMC_',
 'Percentage_of_Active_Physicians_Who_Are_Female__2018__AAMC_',
 'Percentage_of_Active_Physicians_Who_Are_International_Medical_Graduates__IMGs___2018__AAMC_',
 'Percentage_of_Active_Physicians_Who_Are_Age_60_or_Older__2018__AAMC_',
 'MD_and_DO_Student_Enrollment_per_100_000_Population__AY_2018_2019__AAMC_',
 'Student_Enrollment_at_Public_MD_and_DO_Schools_per_100_000_Population__AY_2018_2019__AAMC_',
 'Percentage_Change_in_Student_Enrollment_at_MD_and_DO_Schools__2008_2018__AAMC_',
 'Percentage_of_MD_Students_Matriculating_In_State__AY_2018_2019__AAMC_',
 'Total_Residents_Fellows_in_ACGME_Programs_per_100_000_Population_as_of_December_31__2018__AAMC_',
 'Total_Residents_Fellows_in_Primary_Care_ACGME_Programs_per_100_000_Population_as_of_Dec__31__2018__AAMC_',
 'Percentage_of_Residents_in_ACGME_Programs_Who_Are_IMGs_as_of_December_31__2018__AAMC_',
 'Ratio_of_Residents_and_Fellows__GME__to_Medical_Students__UME___AY_2017_2018__AAMC_',
 'Percent_Change_in_Residents_and_Fellows_in_ACGME_Accredited_Programs__2008_2018__AAMC_',
 'Percentage_of_Physicians_Retained_in_State_from_Undergraduate_Medical_Education__UME___2018__AAMC_',
 'All_Specialties__AAMC_',
 'State_Local_Government_hospital_beds_per_1000_people__2019_',
 'Non_profit_hospital_beds_per_1000_people__2019_',
 'For_profit_hospital_beds_per_1000_people__2019_',
 'Total_hospital_beds_per_1000_people__2019_',
 'Total_nurse_practitioners__2019_',
 'Total_physician_assistants__2019_',
 'Total_Hospitals__2019_',
 'Total_Primary_Care_Physicians__2019_',
 'Surgery_specialists__2019_',
 'Emergency_Medicine_specialists__2019_',
 'Total_Specialist_Physicians__2019_',
 'ICU_Beds',
 'pop_fraction',
 'Length_of_Life_rank',
 'Quality_of_Life_rank',
 'Health_Behaviors_rank',
 'Clinical_Care_rank',
 'Social___Economic_Factors_rank',
 'Physical_Environment_rank',
 'Adult_smoking_percentage',
 'Adult_obesity_percentage',
 'Excessive_drinking_percentage',
 'Population_per_sq_mile',
 'House_per_sq_mile',
 'Share_of_Tests_with_Positive_COVID_19_Results',
 'Number_of_Tests_with_Results_per_1_000_Population']

In [22]:
from pyspark.sql.functions import col

dataset = zeroinf_analysis.select( col('length_of_lockdown'), col('confirmed_cases'), col('confirmed_deaths'),col('POP_ESTIMATE_2018'), col('POP_ESTIMATE_2018'), col('ICU_Beds'), col('Adult_obesity_percentage'),col('Quality_of_Life_rank'), col('Excessive_drinking_percentage'),col('Population_per_sq_mile'), col('Clinical_Care_rank'), col('ICU_Beds'), col('Adult_smoking_percentage'), col('Total_Specialist_Physicians__2019_'), col('Physical_Environment_rank'), col('Number_of_Tests_with_Results_per_1_000_Population'))

In [23]:

dataset.show(5)

+------------------+---------------+----------------+-----------------+-----------------+-----------+------------------------+--------------------+-----------------------------+----------------------+------------------+-----------+------------------------+----------------------------------+-------------------------+-------------------------------------------------+
length_of_lockdown|confirmed_cases|confirmed_deaths|POP_ESTIMATE_2018|POP_ESTIMATE_2018| ICU_Beds|Adult_obesity_percentage|Quality_of_Life_rank|Excessive_drinking_percentage|Population_per_sq_mile|Clinical_Care_rank| ICU_Beds|Adult_smoking_percentage|Total_Specialist_Physicians__2019_|Physical_Environment_rank|Number_of_Tests_with_Results_per_1_000_Population|
+------------------+---------------+----------------+-----------------+-----------------+-----------+------------------------+--------------------+-----------------------------+----------------------+------------------+-----------+------------------------+----------------------------------+-------------------------+-------------------------------------------------+
 26| 93| 4| 55601| 55601|0.107911728| 33| 11.0| 15| 91.8| 13.0|0.107911728| 18| 1.297497418| 53.0| 27.8|
 26| 231| 7| 218022| 218022|0.233921347| 31| 2.0| 18| 114.6| 6.0|0.233921347| 17| 1.297497418| 5.0| 27.8|
 26| 69| 1| 24881| 24881|0.200956553| 42| 55.0| 13| 31.0| 41.0|0.200956553| 22| 1.297497418| 28.0| 27.8|
 26| 46| 1| 22400| 22400| 0.0| 38| 14.0| 16| 36.8| 18.0| 0.0| 19| 1.297497417| 22.0| 27.8|
 26| 45| 0| 57840| 57840| 0.10373444| 34| 13.0| 14| 88.9| 45.0| 0.10373444| 19| 1.297497418| 49.0| 27.8|
+------------------+---------------+----------------+-----------------+-----------------+-----------+------------------------+--------------------+-----------------------------+----------------------+------------------+-----------+------------------------+----------------------------------+-------------------------+-------------------------------------------------+
only showing top 5 rows

In [24]:
dataset.dtypes

Out[239]: [('length_of_lockdown', 'int'),
 ('confirmed_cases', 'int'),
 ('confirmed_deaths', 'int'),
 ('POP_ESTIMATE_2018', 'int'),
 ('POP_ESTIMATE_2018', 'int'),
 ('ICU_Beds', 'double'),
 ('Adult_obesity_percentage', 'int'),
 ('Quality_of_Life_rank', 'double'),
 ('Excessive_drinking_percentage', 'int'),
 ('Population_per_sq_mile', 'double'),
 ('Clinical_Care_rank', 'double'),
 ('ICU_Beds', 'double'),
 ('Adult_smoking_percentage', 'int'),
 ('Total_Specialist_Physicians__2019_', 'double'),
 ('Physical_Environment_rank', 'double'),
 ('Number_of_Tests_with_Results_per_1_000_Population', 'double')]

In [25]:
from pyspark.sql.functions import col


dataset = dataset.withColumnRenamed('confirmed_deaths', 'label')

In [27]:
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor, GeneralizedLinearRegression, AFTSurvivalRegression
from pyspark.ml.feature import VectorIndexer

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import DoubleType
 
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder  

In [28]:
valuableColumns = list(dataset.columns)

In [29]:
valuableColumns

Out[243]: ['length_of_lockdown',
 'confirmed_cases',
 'label',
 'POP_ESTIMATE_2018',
 'POP_ESTIMATE_2018',
 'ICU_Beds',
 'Adult_obesity_percentage',
 'Quality_of_Life_rank',
 'Excessive_drinking_percentage',
 'Population_per_sq_mile',
 'Clinical_Care_rank',
 'ICU_Beds',
 'Adult_smoking_percentage',
 'Total_Specialist_Physicians__2019_',
 'Physical_Environment_rank',
 'Number_of_Tests_with_Results_per_1_000_Population']

In [30]:
for col in valuableColumns[:-1]:
    # Of cause we can't change immutable values, but we can owerwrite them
    dataset = dataset.withColumn(col+"_d", dataset[col].cast("double"))
    
    
dataset = dataset.fillna(-1., subset=valuableColumns)  
dataset

Out[244]: DataFrame[length_of_lockdown: int, confirmed_cases: int, label: int, POP_ESTIMATE_2018: int, POP_ESTIMATE_2018: int, ICU_Beds: double, Adult_obesity_percentage: int, Quality_of_Life_rank: double, Excessive_drinking_percentage: int, Population_per_sq_mile: double, Clinical_Care_rank: double, ICU_Beds: double, Adult_smoking_percentage: int, Total_Specialist_Physicians__2019_: double, Physical_Environment_rank: double, Number_of_Tests_with_Results_per_1_000_Population: double, length_of_lockdown_d: double, confirmed_cases_d: double, label_d: double, POP_ESTIMATE_2018_d: double, ICU_Beds_d: double, Adult_obesity_percentage_d: double, Quality_of_Life_rank_d: double, Excessive_drinking_percentage_d: double, Population_per_sq_mile_d: double, Clinical_Care_rank_d: double, Adult_smoking_percentage_d: double, Total_Specialist_Physicians__2019__d: double, Physical_Environment_rank_d: double]

In [31]:
#let add few train model 
gbt = GBTRegressor(maxIter=100, 
                   maxDepth=5)\
                  .setLabelCol("label")\
                  .setFeaturesCol("features")


indexers = []
indexers.append(VectorAssembler(
    inputCols=["{0}_d".format(col) for col in valuableColumns[:-1]], 
        outputCol="features"))
indexers.append(gbt)

In [32]:
(trainingData, subtestData) = dataset.randomSplit([0.7, 0.3])



In [33]:
modelGBT = Pipeline(stages=tuple(indexers)).fit(trainingData)

In [34]:
predictions = modelGBT.transform(subtestData)

predictions.select(["prediction", "label", "features"]).show(7)

+--------------------+-----+--------------------+
 prediction|label| features|
+--------------------+-----+--------------------+
 0.03744741943873157| 0|[0.0,1.0,0.0,1392...|
-0.02306062930929...| 0|[0.0,1.0,0.0,2153...|
0.009716368215848769| 0|[0.0,1.0,0.0,2742...|
0.004501841761412434| 0|[0.0,1.0,0.0,7450...|
 0.03233951677823164| 0|[0.0,1.0,0.0,1017...|
-0.01136678109213568| 0|[0.0,1.0,0.0,1031...|
-0.01811749745083248| 0|[0.0,1.0,0.0,1077...|
+--------------------+-----+--------------------+
only showing top 7 rows

In [35]:
evaluator = RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = {0}".format(str(rmse)))

Root Mean Squared Error (RMSE) on test data = 55.29591801698357

In [36]:
display(predictions.select("label", "prediction"))

label,prediction
0,0.03744741943873157
0,-0.023060629309297188
0,0.009716368215848769
0,0.004501841761412434
0,0.03233951677823164
0,-0.01136678109213568
0,-0.01811749745083248
0,0.05058571292743506
0,0.020023446831510677
0,0.01701754924720879


In [37]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
grid = ParamGridBuilder() \
  .addGrid(dtc.maxDepth, [2, 3, 4, 5, 6, 7, 8]) \
  .addGrid(dtc.maxBins, [2, 4, 8]) \
  .build()


--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2835302286394087> in <module> 
 3 from pyspark . ml import Pipeline
 4 grid = ParamGridBuilder ( ) \ 
 ----> 5 . addGrid ( dtc . maxDepth , [ 2 , 3 , 4 , 5 , 6 , 7 , 8 ] ) \ 
 6 . addGrid ( dtc . maxBins , [ 2 , 4 , 8 ] ) \ 
 7 . build ( ) 

 NameError : name 'dtc' is not defined

In [38]:
cv = CrossValidator(estimator=pipeline, evaluator=evaluator, estimatorParamMaps=grid, numFolds=3)


In [39]:
# Explicitly create a new run.
# This allows this cell to be run multiple times.
# If you omit mlflow.start_run(), then this cell could run once,
# but a second run would hit conflicts when attempting to overwrite the first run.
import mlflow
import mlflow.mleap
with mlflow.start_run():
  modelGBT = cv.fit(trainingData)
  #modelGBT = Pipeline(stages=tuple(indexers)).fit(trainingData)
  mlflow.set_tag('owner_team', 'UX Data Science') # Logs user-defined tags
  test_metric = evaluator.evaluate(modelGBT.transform(subtestData))
  mlflow.log_metric('subtestData_' + evaluator.getMetricName(), test_metric) # Logs additional metrics
  mlflow.mleap.log_model(spark_model=modelGBT.bestModel, sample_input=subtestData, artifact_path='best-model') # Logs the best model via mleap


--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2130250484355809> in <module> 
 6 import mlflow . mleap
 7 with mlflow . start_run ( ) : 
 ----> 8 modelGBT = cv . fit ( trainingData ) 
 9 #modelGBT = Pipeline(stages=tuple(indexers)).fit(trainingData) 
 10 mlflow . set_tag ( 'owner_team' , 'UX Data Science' ) # Logs user-defined tags 

 NameError : name 'cv' is not defined